In [2]:
%pip install 'chronos-forecasting>=2.0' 'pandas[pyarrow]' 'matplotlib' --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.5/70.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 84.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.2 MB/s eta 0:00:00


In [3]:
import os

# Use only 1 GPU if available
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ["HF_TOKEN"] =  ...

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from chronos import BaseChronosPipeline, Chronos2Pipeline
from IPython.display import clear_output

# Load the Chronos-2 pipeline
# GPU recommended for faster inference, but CPU is also supported
pipeline: Chronos2Pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-2", 
    device_map="cuda"
        # device_map="cpu"
        )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/478M [00:00<?, ?B/s]

In [4]:
df = pd.read_csv('https://www.dropbox.com/scl/fi/s83jig4zews1xz7vhezui/allDataWithCalculatedColumns.csv?rlkey=9mm4zwaugxyj2r4ooyd39y4nl&raw=1')
df.ds = pd.to_datetime(df.ds, errors="coerce")
df['id']='jgh'
df.tail()

,ds,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,...,RAZ_IMCONS_MORE4H,RAZ_XRAY_MORE2H,RAZ_CT_MORE2H1,PSYCH1,PSYCH_WAITINGADM,total_tbs,vert_tbs,pod_tbs,overflow,id
42861,2025-11-25 06:00:00,5,20,1,7,6,27,1,3,0,...,1,0,0,14,9,14,10,4,10,jgh
42862,2025-11-25 07:00:00,1,21,1,8,2,29,0,3,0,...,1,0,0,14,9,13,9,4,10,jgh
42863,2025-11-25 08:00:00,4,25,2,10,6,35,2,5,0,...,1,0,0,15,9,12,10,2,9,jgh
42864,2025-11-25 09:00:00,11,36,9,19,20,55,0,5,0,...,1,0,0,15,9,11,10,1,9,jgh
42865,2025-11-25 10:00:00,7,43,8,27,15,70,3,8,0,...,1,0,0,16,9,15,12,3,9,jgh


In [5]:
all_shifts_df = pd.read_csv('https://www.dropbox.com/scl/fi/yeyr2a7pj6nry8i2q3m0c/all_shifts.csv?rlkey=q1su2h8fqxfnlu7t1l2qe1w0q&raw=1')
all_shifts_df.head()

,scheduled_shift_id,group_id,user_id,employee_id,npi,first_name,last_name,facility_id,facility_ext_id,facility_name,...,shift_short_name,shift_start,shift_end,shift_hours,work_start,work_end,work_hours,count_as_shift,is_night,is_weekend
0,9021,1,40,NaN,NaN,Joel,Turner,1,NaN,Jewish General Hospital,...,W1,2021-01-01 08:00:00,2021-01-01 16:00:00,8.0,1/1/2021 08:00,1/1/2021 16:00,8.0,0,0,0
1,9022,1,52,NaN,NaN,Gregory,Marton,1,NaN,Jewish General Hospital,...,X1,2021-01-01 08:00:00,2021-01-01 17:00:00,9.0,1/1/2021 08:00,1/1/2021 17:00,9.0,1,0,0
2,9023,1,20,NaN,NaN,Will,Grad,1,NaN,Jewish General Hospital,...,X3,2021-01-01 08:00:00,2021-01-01 17:00:00,9.0,1/1/2021 08:00,1/1/2021 17:00,9.0,1,0,0
3,9025,1,17,NaN,NaN,Jerry,Dankoff,1,NaN,Jewish General Hospital,...,X4,2021-01-01 08:00:00,2021-01-01 16:00:00,8.0,1/1/2021 08:00,1/1/2021 16:00,8.0,1,0,0
4,9026,1,26,NaN,NaN,Marie Renee,Lajoie,1,NaN,Jewish General Hospital,...,X2,2021-01-01 08:00:00,2021-01-01 16:00:00,8.0,1/1/2021 08:00,1/1/2021 16:00,8.0,1,0,0


In [ ]:
shift_types_dict = {
    
}


In [ ]:
all_shifts_df.